In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
ALS_norm = pd.read_csv("C:/PhD_study/Programming/python37/GEOG770/data/04_ALS_norm_seg.txt",sep=" ",header=None,names=['x','y','ele'])
ALS_max = pd.read_csv("C:/PhD_study/Programming/python37/GEOG770/data/local_max.txt",sep=" ",header=None,names=['x','y','ele'])

In [3]:
def equal_num(a, b):
    temp = abs(a-b)
    if temp < 0.00001:
        return True
    else:
        return False

In [4]:
def get_highest_pt(U):
    index_ele_max_U = U.idxmax()[2] # the 2nd col denotes elevation
    pt_max = pd.Series([U['x'][index_ele_max_U], U['y'][index_ele_max_U], U['ele'][index_ele_max_U]])
    return pt_max, index_ele_max_U
    

In [5]:
# U is used to store the whole points
U = pd.read_csv("C:/PhD_study/Programming/python37/GEOG770/data/04_ALS_norm_seg.txt",sep=" ",header=None,names=['x','y','ele'])

#P = pd.DataFrame(columns=['x','y','ele'])

# P will store points belonging to the highest tree
P = pd.DataFrame()

# N will store points not belonging to the highest tree
N = pd.DataFrame()

# obtain the index of the point with maximum elevation in ALS_norm
# index_ele_max_U = U.idxmax()[2] # the 2nd col denotes elevation
# pt_max = pd.Series([U['x'][index_ele_max_U], U['y'][index_ele_max_U], U['ele'][index_ele_max_U]])
pt_max, pt_max_index = get_highest_pt(U)

# adding the highest point to P
P = P.append({'x':pt_max[0],'y':pt_max[1],'ele':pt_max[2]}, ignore_index=True)

# remove the highest point from U
U.drop(labels=pt_max_index,inplace=True)
U = U.reset_index(drop=True)

In [6]:
# find a point far away from index_ele_max_U and add this point to N
for i in range(len(U)):
    dis = np.sqrt((pt_max[0] - U['x'][i])**2 + (pt_max[1] - U['y'][i])**2)
    if dis > 50:
        N = N.append({'x':U['x'][i],'y':U['y'][i],'ele':U['ele'][i]}, ignore_index=True)
        N.drop_duplicates(keep='first', inplace=True) # drop the duplicates if existing
        U.drop(labels=pt_max_index,inplace=True)
        U = U.reset_index(drop=True)
        break

In [8]:
def min_dis(df, pt):
    dis_min = np.sqrt((pt[0] - df['x'][0])**2 + (pt[1] - df['y'][0])**2)
    if len(df) > 1:
        for i in range(1, len(df)):
            dis = np.sqrt((pt[0] - df['x'][i])**2 + (pt[1] - df['y'][i])**2)
            if dis_min > dis:
                dis_min = dis
    
    return dis_min        

In [9]:
def Max_contain(df, pt):
    for i in range(len(df)):
        if equal_num(df['x'][i],pt[0]) and equal_num(df['y'][i],pt[1]) and equal_num(df['ele'][i],pt[2]):
            return True

In [11]:
t0 = time.time()
dis_thre = 2.0
for i in range(len(U)):
    # us is the highest point in U, uz is a pd.series object
    uz, uz_index = get_highest_pt(U)
    dis_min_P = min_dis(P,uz)
    dis_min_N = min_dis(N,uz)
    # judge if uz is the local maximum point
    if Max_contain(ALS_max, uz):
        if dis_min_P > dis_thre:
            N = N.append({'x':uz[0],'y':uz[1],'ele':uz[2]}, ignore_index=True)
        if dis_min_P <= dis_thre and dis_min_P <= dis_min_N:
            P = P.append({'x':uz[0],'y':uz[1],'ele':uz[2]}, ignore_index=True)
        if dis_min_P <= dis_thre and dis_min_P > dis_min_N:
            N = N.append({'x':uz[0],'y':uz[1],'ele':uz[2]}, ignore_index=True)
    else:
        if dis_min_P <= dis_min_N:
            P = P.append({'x':uz[0],'y':uz[1],'ele':uz[2]}, ignore_index=True)
        if dis_min_P > dis_min_N:
            N = N.append({'x':uz[0],'y':uz[1],'ele':uz[2]}, ignore_index=True)
    
    # remove uz from U
    U.drop(labels=uz_index,inplace=True)
    U = U.reset_index(drop=True)
t1 = time.time()
print("time:",t1-t0)

time: 11481.762405872345


In [12]:
len(U)

0

In [13]:
len(P)

1168

In [14]:
len(N)

30962

In [15]:
P

,ele,x,y
0,2.813000e+01,320551.419998,4.990802e+06
1,2.765000e+01,320551.439999,4.990801e+06
2,2.740000e+01,320551.650002,4.990802e+06
3,2.733000e+01,320551.029999,4.990802e+06
4,2.724000e+01,320551.849999,4.990802e+06
...,...,...,...
1163,4.657054e-08,320549.209999,4.990795e+06
1164,-1.000020e-02,320558.530003,4.990807e+06
1165,-1.999995e-02,320553.890000,4.990801e+06
1166,-5.000014e-02,320552.340000,4.990797e+06


In [16]:
'''
with open("C:/PhD_study/Programming/python37/GEOG770/data/tree_highest.txt", "w") as output:
    for pid in range(len(P)):
        output.write("{} {} {}\n".format(P['x'][pid], P['y'][pid],P['ele'][pid]))
'''

In [ ]:
with open("C:/PhD_study/Programming/python37/GEOG770/data/individual_trees/tree_1_dis_thre80.csv", "w", newline='') as output:
    field_names = ['x_cor', 'y_cor', 'elevation']
    writer = csv.DictWriter(output, fieldnames=field_names)
    # writer.writeheader()
    for pid in range(len(P)):
        #output.write("{} {} {}\n".format({'x_cor':P['x'][pid], 'y_cor':P['y'][pid],'elevation':P['ele'][pid]}))
        writer.writerow({'x_cor':P['x'][pid], 'y_cor':P['y'][pid],'elevation':P['ele'][pid]})

In [17]:
(t1-t0)/3600

3.1893784460756516